## mount the google drive and change the directory

In [15]:
from google.colab import drive
import os

## mount the google drive
drive.mount('/content/drive')
## change the directory to the target teeth segmentation
%cd /content/drive/MyDrive/dilated_tooth_seg_net

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/dilated_tooth_seg_net


## upload and test the .obj file

- In this code, you are required to upload both the data file as well as the json file, so you will be asked to first upload the .obj file and then the .json file

In [17]:
from google.colab import files
import os

# Create an upload directory
upload_directory = 'upload'
if not os.path.exists(upload_directory):
    os.makedirs(upload_directory)

uploaded_files = []
for i in range(2):
    uploaded = files.upload()
    for fn in uploaded.keys():
        destination_path = os.path.join(upload_directory, fn)
        with open(destination_path, 'wb') as f:
            f.write(uploaded[fn])
        uploaded_files.append(destination_path)
        print(f'File "{fn}" uploaded to {destination_path}')

print("\nUploaded files and their destinations:")
for file_path in uploaded_files:
    print(file_path)

Saving 00OMSZGW_lower.obj to 00OMSZGW_lower.obj
File "00OMSZGW_lower.obj" uploaded to upload/00OMSZGW_lower.obj


Saving 00OMSZGW_lower.json to 00OMSZGW_lower.json
File "00OMSZGW_lower.json" uploaded to upload/00OMSZGW_lower.json

Uploaded files and their destinations:
upload/00OMSZGW_lower.obj
upload/00OMSZGW_lower.json


In [18]:
uploaded_files

['upload/00OMSZGW_lower.obj', 'upload/00OMSZGW_lower.json']

### inference

In [23]:
!pip install pyfqmr trimesh lightning polyscope -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 49.2 MB/s eta 0:00:00


In [20]:
## change the sys directory to import custom modules
from sys import path
path.append('/content/drive/MyDrive/dilated_tooth_seg_net')

## Inference

In [24]:
## github scripts
import trimesh
import torch
import json
import pyfqmr
import numpy as np
from pathlib import Path
from models.dilated_tooth_seg_network import LitDilatedToothSegmentationNetwork
from utils.teeth_numbering import color_mesh,colors_to_label,fdi_to_label
from lightning.pytorch import seed_everything
import copy
from scipy import spatial
import random

# same function in mesh_dataset
def process_mesh(mesh: trimesh, labels: torch.tensor = None):
    mesh_faces = torch.from_numpy(mesh.faces.copy()).float()
    mesh_triangles = torch.from_numpy(mesh.vertices[mesh.faces]).float()
    mesh_face_normals = torch.from_numpy(mesh.face_normals.copy()).float()
    mesh_vertices_normals = torch.from_numpy(mesh.vertex_normals[mesh.faces]).float()
    if labels is None:
        labels = torch.from_numpy(colors_to_label(mesh.visual.face_colors.copy())).long()
    return mesh_faces, mesh_triangles, mesh_vertices_normals, mesh_face_normals, labels

# similar function as PreTransform in preprocessing.py
def preporces(data):
    mesh_faces, mesh_triangles, mesh_vertices_normals, mesh_face_normals, labels = data
    mesh = trimesh.Trimesh(**trimesh.triangles.to_kwargs(mesh_triangles.cpu().detach().numpy()))

    points = torch.from_numpy(mesh.vertices)
    v_normals = torch.from_numpy(mesh.vertex_normals)

    s, _ = mesh_faces.size()
    x = torch.zeros(s, 24).float()
    x[:, :3] = mesh_triangles[:, 0]
    x[:, 3:6] = mesh_triangles[:, 1]
    x[:, 6:9] = mesh_triangles[:, 2]
    x[:, 9:12] = mesh_triangles.mean(dim=1)
    x[:, 12:15] = mesh_vertices_normals[:, 0]
    x[:, 15:18] = mesh_vertices_normals[:, 1]
    x[:, 18:21] = mesh_vertices_normals[:, 2]
    x[:, 21:] = mesh_face_normals

    maxs = points.max(dim=0)[0]
    mins = points.min(dim=0)[0]
    means = points.mean(axis=0)
    stds = points.std(axis=0)
    nmeans = v_normals.mean(axis=0)
    nstds = v_normals.std(axis=0)
    nmeans_f = mesh_face_normals.mean(axis=0)
    nstds_f = mesh_face_normals.std(axis=0)
    for i in range(3):
        # normalize coordinate
        x[:, i] = (x[:, i] - means[i]) / stds[i]  # point 1
        x[:, i + 3] = (x[:, i + 3] - means[i]) / stds[i]  # point 2
        x[:, i + 6] = (x[:, i + 6] - means[i]) / stds[i]  # point 3
        x[:, i + 9] = (x[:, i + 9] - mins[i]) / (maxs[i] - mins[i])  # centre
        # normalize normal vector
        x[:, i + 12] = (x[:, i + 12] - nmeans[i]) / nstds[i]  # normal1
        x[:, i + 15] = (x[:, i + 15] - nmeans[i]) / nstds[i]  # normal2
        x[:, i + 18] = (x[:, i + 18] - nmeans[i]) / nstds[i]  # normal3
        x[:, i + 21] = (x[:, i + 21] - nmeans_f[i]) / nstds_f[i]  # face normal

    pos = x[:, 9:12]

    return pos, x, labels

# same function(method) in mesh_dataset.Teeth3DSDataset
def Downsample(mesh,labels):
    mesh_simplifier = pyfqmr.Simplify()
    mesh_simplifier.setMesh(mesh.vertices, mesh.faces)
    mesh_simplifier.simplify_mesh(target_count=16000, aggressiveness=3, preserve_border=True, verbose=0,
                                  max_iterations=2000)
    new_positions, new_face, _ = mesh_simplifier.getMesh()
    mesh_simple = trimesh.Trimesh(vertices=new_positions, faces=new_face)
    vertices = mesh_simple.vertices
    faces = mesh_simple.faces
    if faces.shape[0] < 16000:
        fs_diff = 16000 - faces.shape[0]
        faces = np.append(faces, np.zeros((fs_diff, 3), dtype="int"), 0)
    elif faces.shape[0] > 16000:
        mesh_simple = trimesh.Trimesh(vertices=vertices, faces=faces)
        samples, face_index = trimesh.sample.sample_surface_even(mesh_simple, 16000)
        mesh_simple = trimesh.Trimesh(vertices=mesh_simple.vertices, faces=mesh_simple.faces[face_index])
        faces = mesh_simple.faces
        vertices = mesh_simple.vertices
    mesh_simple = trimesh.Trimesh(vertices=vertices, faces=faces)

    mesh_v_mean = mesh.vertices[mesh.faces].mean(axis=1)
    mesh_simple_v = mesh_simple.vertices
    tree = spatial.KDTree(mesh_v_mean)
    query = mesh_simple_v[faces].mean(axis=1)
    distance, index = tree.query(query)
    labels = labels[index].flatten()
    return mesh_simple,labels

# reverse normalization
def PostProces(data_OG_def,x_def):
    _, mesh_triangles, _, mesh_face_normals, _ = data_OG_def
    mesh = trimesh.Trimesh(**trimesh.triangles.to_kwargs(mesh_triangles.cpu().detach().numpy()))

    maxs = mesh.vertices.max(axis=0)
    mins =  mesh.vertices.min(axis=0)
    means =  mesh.vertices.mean(axis=0)
    stds =  mesh.vertices.std(axis=0)
    nmeans = mesh.vertex_normals.mean(axis=0)
    nstds = mesh.vertex_normals.std(axis=0)
    nmeans_f = mesh_face_normals.mean(axis=0)
    nstds_f = mesh_face_normals.std(axis=0)
    for i in range(3):
        #  coordinate
        x_def[:, i] = (x_def[:, i] + means[i]) * stds[i]  # point 1
        x_def[:, i + 3] = (x_def[:, i + 3] + means[i]) * stds[i]  # point 2
        x_def[:, i + 6] = (x_def[:, i + 6] + means[i]) * stds[i]  # point 3
        x_def[:, i + 9] = (x_def[:, i + 9] + mins[i]) * (maxs[i] - mins[i])  # centre
        #  normal vector
        x_def[:, i + 12] = (x_def[:, i + 12] + nmeans[i]) * nstds[i]  # normal1
        x_def[:, i + 15] = (x_def[:, i + 15] + nmeans[i]) * nstds[i]  # normal2
        x_def[:, i + 18] = (x_def[:, i + 18] + nmeans[i]) * nstds[i]  # normal3
        x_def[:, i + 21] = (x_def[:, i + 21] + nmeans_f[i]) * nstds_f[i]  # face normal
    return x_def



In [26]:
!ls

00OMSZGW_lower.json  logs    README.md	       test.py			      upload
00OMSZGW_lower.obj   models  requirements.txt  test_teeth_segmentation.ipynb  utils
dataset		     output  test_network.py   train_network.py		      visualize_example.py


In [29]:
SEED = 42
use_gpu=True
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)
torch.set_float32_matmul_precision('medium')
random.seed(SEED)
seed_everything(SEED, workers=True)

#-----Define values
Model_Teeth= uploaded_files[0] # .obj file path in Teeth3DS dataset example: Teeth3DS\Upper\\0JN50XQR\\0JN50XQR_upper.obj
json_file = uploaded_files[1]
ML_parameters=  "logs/training/1/epoch=89-step=54000.ckpt" # model parameter file path
out_dir = 'output'

output_filename = Model_Teeth.split('/')[-1].split('.')[0]

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

#----------Model----------
model = LitDilatedToothSegmentationNetwork.load_from_checkpoint(ML_parameters)
if use_gpu==True:
   model = model.cuda()

#----Import model
mesh=trimesh.load(Path(Model_Teeth))

with open(json_file) as f:
     data = json.load(f)
labels = np.array(data["labels"])
labels = labels[mesh.faces]
labels = labels[:, 0]
labels = fdi_to_label(labels)

#----Downsample
mesh_simple,labels=Downsample(mesh,labels)

#----Preporcess
data = process_mesh(mesh_simple, torch.from_numpy(labels).long())
data_OG=copy.copy(data)
data =preporces(data)

#----Ground truth model labels
ground_truth = data[2]
mesh_gt = color_mesh(mesh_simple, ground_truth.numpy())
## removing the gum from the ground truth mesh
mesh_gt = mesh_gt.submesh([np.where(ground_truth.numpy() != 0)[0]], append=True)
mesh_gt.export(os.path.join(out_dir, f'{output_filename}_gt.ply')) # export ground truth 3D model

#----Use model
pre_labels = model.predict_labels(data).cpu().numpy()
x=PostProces(data_OG,data[1]) # Postprocess

triangles = x[:, :9].reshape(-1, 3, 3)
mesh = trimesh.Trimesh(**trimesh.triangles.to_kwargs(triangles.cpu().detach().numpy()))
mesh_pred = color_mesh(mesh, pre_labels)

## removing the gum from the mesh prediction
mesh_pred = mesh_pred.submesh([np.where(pre_labels != 0)[0]], append=True)

mesh_pred.export(os.path.join(out_dir, f'{output_filename}_pred.ply')) # export predicted 3D model
print('Done')


INFO: Seed set to 42
INFO:lightning.fabric.utilities.seed:Seed set to 42


Done


In [30]:
mesh_gt.show()

In [31]:
mesh_pred.show()